In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

nba_ = pd.read_csv("/Users/ncapofari/Documents/CUNY_SPS/Capstone/%s" 
                  % "nba_scraper/data_sets/4factor_model_data.csv")

nba = nba_[nba_['Seed_Diff'] > 0]
nba = nba_[nba_['Seed_Diff'] > 1]
nba = nba_[nba_['Seed_Diff'] == 1]

train_set, test_set = train_test_split(nba, 
                                       test_size=0.2, 
                                       random_state=23)

In [54]:
#models

nba_train = train_set["Seed_Diff"].copy().values.reshape(-1, 1)
nba_train = train_set["Pyt_Win_Pct_Diff"].copy().values.reshape(-1, 1)
nba_train = train_set.drop(["Seed_Diff","Pyt_Win_Pct_Diff",
                            "Won_Series", "id"], axis=1)

f = "four_fact_KNN_is_one"

nba_labels = train_set["Won_Series"].copy()
nba_labels = (nba_labels == 1)
nba_test = test_set.copy()

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
    #('select', SelectKBest(f_classif, 5))
])

nba_prepared = pipeline.fit_transform(nba_train, nba_labels)
print(nba_prepared.shape)

(118, 15)


In [56]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn_clf = KNeighborsClassifier()
param_grid = {'n_neighbors': [2,4,6,8,10,12],
              'weights': ['uniform','distance'], }
knn_clf = GridSearchCV(knn_clf, param_grid, cv=5)
knn_clf.fit(nba_prepared, nba_labels)

print("Best estimator found by grid search:")
print(knn_clf.best_estimator_)

Best estimator found by grid search:
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')


In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.metrics import roc_curve, roc_auc_score

def scores(m, fs, tgt, fname):
    cross_val_score(m, fs, tgt, cv=5)
    y_train_pred = cross_val_predict(m, fs, tgt, cv=5)
    print(confusion_matrix(tgt, y_train_pred))
    print("precision score:", precision_score(tgt, y_train_pred))
    fpr, tpr, thresholds = roc_curve(tgt, y_train_pred)
    print("roc_auc:", roc_auc_score(tgt, y_train_pred))
    temp = pd.DataFrame({"fpr":fpr, "tpr":tpr, "thresholds":thresholds})
    temp.to_csv("scores/"+fname+".csv")
    preds = pd.DataFrame({"Prediction":y_train_pred})
    x = pd.concat([preds.reset_index(drop=True), 
                   train_set.reset_index(drop=True)], axis=1)
    x.to_csv("results/"+fname+".csv")

In [58]:
scores(knn_clf, nba_prepared, nba_labels, f)

[[42 23]
 [38 15]]
precision score: 0.39473684210526316
roc_auc: 0.46458635703918716
